In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from simulation.guru_tester import GuruTester

In [3]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [4]:
def run_pair(pair):
    df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = RSI(df_an)
    df_an = apply_patterns(df_an)
    df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
            'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True
    )
    
    gt.run_test()
    return gt.df_results
    

In [5]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p)))

prepare_data...
run_test...
Result: -2065.5
prepare_data...
run_test...
Result: -915.0
prepare_data...
run_test...
Result: -1109.5
prepare_data...
run_test...
Result: -1200.5


In [6]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -2065.5
EUR_USD -915.0
GBP_JPY -1109.5
USD_CAD -1200.5
